In [1]:
# The purpose of the following project is to get some insight on spirit prices. 
# This notebook contains a parser whose main objecetive is to parse the data from "liquorama", an online brewery
# store from California. This script will extract information about their beer listings and obtain data regarding characteristics
# such as price, brand, reviews ETCETERA. It will store the resulting objects into a JSON dictionary that will be transformed
# in a pandas dataframe in 


#First it will import the required libraries in order to scrape the data succesfully

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen, Request
import json

In [3]:
print(requests.get("https://www.liquorama.net/robots.txt").text)

User-agent: *
Disallow: /search.php
User-agent: *
Disallow: /*?_bc_fsnf=1*
Disallow: /*&_bc_fsnf=1*



In [ ]:
#The connection to the website was succesful and allows for scraping

In [ ]:
################################################

# Disallow: /?_bc_fsnf=1 — This blocks bots from following faceted search links and causing performance issues.

# Disallow: /&_bc_fsnf=1 — This blocks bots from following faceted search links and causing performance issues.

# Disallow: /search.php — This page handles searches from the search box on a store. Google has previously stated 
#                        that search results pages are not something they want in their index because it creates a poor 
#                        user experience (going from a search results page to another search results page instead of going 
#                        directly to the result).
#
#
# Source: Big Commerce (Help Center), "Understanding the Robots.txt file", 
# https://support.bigcommerce.com/s/article/Understanding-the-Robots-txt-File
################################################

In [ ]:
# The purpose of this scraper is to get the general information of Liquorama's beer listing. The online store has a 21 page listing 
# of beers for different size, price, country of origin, reviews and type. 

# Firstly, we need the complete set of links in an object for further parsing.

In [96]:
def GetSoup(link):
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text,'lxml')
    

In [303]:
def GetLinks(link):                     
    soup = GetSoup(link)
    l = []
    for link in soup.findAll('a', attrs={"class":"product-images"}):
        l.append(link.get('href'))
    return l

In [304]:
booze = GetLinks("https://uptownspirits.com/shop/liquor/?product_count=48")

In [136]:
sites = []
for cat in categories: 
       for i in range (1,7):
        sites.append(cat + "?sort=featured&page=" + str(i))

In [307]:
prueba = booze[:10]

In [366]:
booze

['https://uptownspirits.com/product/liquor/whiskey/american-whiskey/skrewball-peanut-butter-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/bourbon-whiskey/e-h-taylor-small-batch-bourbon-whiskey/',
 'https://uptownspirits.com/product/liquor/whiskey/eagle-rare-bourbon-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/liqueur/pennsylvania-dutch-egg-nog-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/buffalo-trace-bourbon-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/rye-whiskey/willett-family-estate-bottled-4-year-rye-750ml/',
 'https://uptownspirits.com/product/liquor/gin/empress-1908-gin-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/bourbon-whiskey/blantons-single-barrel-bourbon-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/sazerac-rye-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/tequila/reposado/tequila-clase-azul-reposado-750ml/',
 'https://uptownspirits.com/product/liquor/tequi

In [ ]:
#Afterwards we must build a function that allows us to convert an html from the website to a BeautifulSoup object (BSO)

In [200]:
def parse_dict(link):
    soup = GetSoup(link)
    return div = soup.find("div", {'class':'card-body purchase-ability withoutTax'})
    return {
                "name":div.find('h4', {'class':'card-title'}).text,
                "price":soup.find('span', {'class':'price price--withoutTax'}).text,
                "previous_price":soup.find('span', {'class':'price price--rrp'}).text,
                "category":soup.find('h2', {'class':'container-header'}).string
                #Change sibling
        }

SyntaxError: invalid syntax (<ipython-input-200-347118b84c36>, line 3)

In [147]:
l=[]
for site in sites:
    soupObj = parse_dict(link)
    l.append(soupObj)
df = pd.DataFrame(l)

In [199]:
parse_dict("https://www.liquorama.net/apertifs-digestives-vermouths/?sort=featured&page=5")

AttributeError: 'NoneType' object has no attribute 'find'

In [79]:
##### HOW CAN I ADD ANOTHER ELEMENT WITHOUT SCARIFICING TIME AND MEMORY?

[]

In [ ]:
##### triunfando en OOP

In [342]:
class Scraper:
    
    def __init__(self, url):
        self.url= url
        
    def GetSoup(self):
        r = requests.get(self.url)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text,'lxml')
         
    def ParseSKU(self):
        soup = GetSoup(self.url)
        return soup.find('span', attrs={'class':'sku'}).text
    
    def ParseName(self):
        soup = GetSoup(self.url)
        return soup.find('h1', attrs={'class':'product_title entry-title'}).text

    def ParsePrice(self):
        soup = GetSoup(self.url)
        return soup.find('span', attrs={'class':'woocommerce-Price-amount amount'}).text
    
    def ParseBrand(self):
        soup = GetSoup(self.url)
        return soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_brand'}).text
    
    def ParseAvailability(self):
        soup = GetSoup()
        return soup.find('div', attrs={'class':'avada-availability'}).text
    
    def Region(self):
        soup = GetSoup(link)
        el = soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_region'}).text
        return el.replace("Region","")
    
    def ParseWeight(self):
        soup = GetSoup()
        el = soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--weight'}).text
        return el.replace('Weight', '')
    
    
            


In [343]:
test = Scraper("https://uptownspirits.com/product/liquor/whiskey/japanese-whiskey/nikka-from-the-barrel-whiskey-750ml/")
type(test)

__main__.Scraper

In [338]:
test.ParsePrice()

'$119.95'

In [340]:
test.ParseSKU()

'4904230049739'

In [344]:
test.ParseBrand()

'BrandNikka'

In [312]:
prueba

['https://uptownspirits.com/product/liquor/whiskey/american-whiskey/skrewball-peanut-butter-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/bourbon-whiskey/e-h-taylor-small-batch-bourbon-whiskey/',
 'https://uptownspirits.com/product/liquor/whiskey/eagle-rare-bourbon-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/liqueur/pennsylvania-dutch-egg-nog-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/buffalo-trace-bourbon-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/rye-whiskey/willett-family-estate-bottled-4-year-rye-750ml/',
 'https://uptownspirits.com/product/liquor/gin/empress-1908-gin-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/bourbon-whiskey/blantons-single-barrel-bourbon-750ml/',
 'https://uptownspirits.com/product/liquor/whiskey/sazerac-rye-whiskey-750ml/',
 'https://uptownspirits.com/product/liquor/tequila/reposado/tequila-clase-azul-reposado-750ml/']

In [202]:
##### parte 3 #### 

In [207]:
whisky = []
for i in range (1,4):
    whisky.append("https://uptownspirits.com/shop/liquor/whiskey/page/"+ str(i) +"/?product_count=144")

In [208]:
whisky

['https://uptownspirits.com/shop/liquor/whiskey/page/1/?product_count=144',
 'https://uptownspirits.com/shop/liquor/whiskey/page/2/?product_count=144',
 'https://uptownspirits.com/shop/liquor/whiskey/page/3/?product_count=144']

In [367]:
def ParseName(link):
    soup = GetSoup(link)
    return soup.find('h1', attrs={'class':'product_title entry-title'}).text
[ParseName(link) for link in booze]

['Skrewball Peanut Butter Whiskey 750ml',
 'E.H. Taylor Small Batch Bourbon Whiskey 750ml',
 'Eagle Rare Bourbon Whiskey 750ml',
 'Pennsylvania Dutch Egg Nog 750ml',
 'Buffalo Trace Bourbon Whiskey 750ml',
 'Willett Family Estate Bottled 4 Year Rye 750ml',
 'Empress 1908 Gin 750ml',
 'Blanton’s Single Barrel Bourbon 750ml',
 'Sazerac Rye Whiskey 750ml',
 'Tequila Clase Azul Reposado 750ml',
 'Cava De Oro Extra Anejo Tequila 750ml',
 'Woodford Reserve 2019 20th Anniversary Kentucky Derby Bourbon Whiskey',
 'Puerto Vallarta Gold Tequila 750ml',
 'Michters American Whiskey 750ml',
 'Buffalo Trace Bourbon Cream Liqueur 750ml',
 'Ciroc Summer Watermelon Vodka',
 'Four Roses Single Barrel Straight Bourbon Whiskey 750ml',
 'Willett Pot Still Reserve Small Batch Bourbon Whiskey',
 'Bumbu XO Rum',
 'Weller Special Reserve Bourbon Whiskey 750ml',
 'Proper No. Twelve Irish Whiskey 750ml',
 'Old Weller Antique 107 Bourbon Whiskey 750ml',
 'Remy Martin Just Remy Limited Edition 1738 Sneaker Box',
 

In [368]:
def ParsePrice(link):
    soup = GetSoup(link)
    el = soup.find('span', attrs={'class':'woocommerce-Price-amount amount'})
    return el.text if el else "NA"
[ParsePrice(link) for link in booze]

['$24.95',
 '$49.95',
 '$33.95',
 '$9.95',
 '$24.95',
 '$59.95',
 '$29.95',
 '$139.95',
 '$27.95',
 '$84.95',
 '$84.95',
 '$54.95',
 '$10.95',
 '$33.33',
 '$18.95',
 '$26.95',
 '$49.95',
 '$49.95',
 '$38.95',
 '$59.95',
 '$24.95',
 '$109.95',
 '$59.95',
 '$88.95',
 '$59.95',
 '$49.95',
 '$34.95',
 '$39.95',
 '$49.95',
 '$34.95',
 '$69.95',
 '$3.95',
 '$249.95',
 '$144.95',
 '$16.95',
 '$129.95',
 '$89.95',
 '$11.95',
 '$19.95',
 '$99.95',
 '$29.95',
 '$44.95',
 '$119.95',
 '$88.95',
 '$59.95',
 '$199.95',
 '$149.95',
 '$49.95']

In [369]:
def ParseSKU(link):
    soup = GetSoup(link)
    el = soup.find('span', attrs={'class':'sku'})
    return el.text if el else "NA"
[ParseSKU(link) for link in booze]

['860265002405',
 '088004005498',
 '088004021344',
 '084380360947',
 '080244009236',
 'NA',
 '628451773108',
 'NA',
 '088004139940',
 '081240049516',
 '002978210062',
 '081128022280',
 'NA',
 '039383005944',
 '008024400371',
 '088076183728',
 '040063400027',
 '680203019353',
 '813497006147',
 '088004025748',
 '811538019569',
 '088004025649',
 'NA',
 '850047003065',
 'NA',
 '853763002244',
 '813497006109',
 'NA',
 '082000782919',
 '088076182783',
 '088352131238',
 '088004022693',
 '080686934141',
 '085834900184',
 '080686001409',
 '854396005190',
 '7502221968105',
 '080686831013',
 '619947000020',
 '080686934073',
 '096749031559',
 '752830059290',
 '4904230049739',
 '500462214333',
 '088352133362',
 '088857001616',
 '080244006839',
 '850047003003']

In [370]:
def ParseWeight(link):
    soup = GetSoup(link)
    el = soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--weight'}).text
    return el.replace('Weight', '')
[ParseWeight(link) for link in booze]

['3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '7 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '8 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '1.5 lbs',
 '3.5 lbs',
 '5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs',
 '3.5 lbs']

In [371]:
def ParseBrand(link):
    soup = GetSoup(link)
    el = soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_brand'})
    ele = el.text if el else "NA"
    return ele.replace("Brand", "")
[ParseBrand(link) for link in booze]

['Skrewball',
 'E.H. Taylor',
 'Buffalo Trace, Eagle Rare',
 'Pennsylvania Dutch',
 'Buffalo Trace',
 'Willett',
 'Empress 1908',
 'NA',
 'Sazerac',
 'Clase Azul',
 'Cava De Oro',
 'Woodford Reserve',
 'Puerto Vallarta',
 'Mitchers',
 'Buffalo Trace',
 'Ciroc',
 'NA',
 'Willett',
 'Bumbu',
 'Weller',
 'Proper No Twelve',
 'Weller',
 'Remy Martin',
 'Angels Envy',
 'Four Roses',
 'MB Roland',
 'Bumbu Rum Co',
 'Henry McKenna',
 'Crown Royal',
 'Johnnie Walker',
 'Rebel Yell',
 'Buffalo Trace',
 'Hibiki',
 'Tequila Dos Artes',
 'Jim Beam',
 'High West',
 'Fortaleza',
 'Sauza',
 'Titos',
 'Hibiki',
 'Larceny',
 'Blackened',
 'Nikka',
 'Adictivo',
 'Old Ezra',
 'Yamazaki',
 'Rock Hill Farms',
 "Angel's Envy"]

In [373]:
def ParseAlcType(link):
    soup = GetSoup(link)
    el = soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_alcohol-type'})
    ele = el.text if el else "NA"
    return ele.replace('Alcohol Type', "")
[ParseAlcType(link) for link in booze]

['Whiskey',
 'Whiskey',
 'Whiskey',
 'Liqueur',
 'Whiskey',
 'Rye',
 'Gin',
 'NA',
 'Whiskey',
 'Tequila',
 'Tequila',
 'Whiskey',
 'Whiskey',
 'Whiskey',
 'Liqueur',
 'Vodka',
 'NA',
 'Whiskey',
 'Rum',
 'Whiskey',
 'Whiskey',
 'Whiskey',
 'Cognac',
 'Whiskey',
 'Whiskey',
 'Whiskey',
 'Rum',
 'NA',
 'Whiskey',
 'Scotch',
 'Whiskey',
 'Whiskey',
 'Whiskey',
 'Tequila',
 'Scotch',
 'Whiskey',
 'Tequila',
 'Tequila',
 'Tequila',
 'Whiskey',
 '[email\xa0protected]',
 'Whiskey',
 'Whiskey',
 'Tequila',
 'Whiskey',
 'Whiskey',
 'NA',
 'Whiskey']

In [374]:
def Class(link):
    soup = GetSoup(link)
    el = soup.find('tr', attrs={'class':"woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_class"})
    ele = el.text if el else "NA"
    return ele.replace("Class","")
[Class(link) for link in booze]

['Peanut Butter Whiskey',
 'Bourbon Whiskey',
 'Bourbon Whiskey',
 'NA',
 'NA',
 'American Whiskey',
 'NA',
 'NA',
 'Rye Whiskey',
 'Reposado',
 'Extra Anejo',
 'NA',
 'NA',
 'NA',
 'Bourbon Liqueur',
 'NA',
 'NA',
 'American Whiskey',
 'NA',
 'Bourbon Whiskey',
 'NA',
 'Bourbon Whiskey',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'Bourbon Whiskey',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'Japanese Whisky',
 'NA',
 'NA',
 'Japanese Whisky',
 'Extra Anejo',
 'NA',
 'Japanese Whisky',
 'Bourbon Whiskey',
 'NA']

In [375]:
def Region(link):
    soup = GetSoup(link)
    el = soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_region'})
    ele = el.text if el else "NA"
    return ele.replace("Region","")
[Region(link) for link in booze]

['America',
 'America',
 'America',
 'NA',
 'NA',
 'America',
 'Canada',
 'America',
 'America',
 'Mexico',
 'Mexico',
 'America',
 'NA',
 'NA',
 'NA',
 'France',
 'NA',
 'America',
 'Barbados',
 'America',
 'Ireland',
 'America',
 'France',
 'America',
 'America',
 'America',
 'Barbados',
 'America',
 'Canada',
 'Scotland',
 'America',
 'NA',
 'Japan',
 'Mexico',
 'NA',
 'America',
 'Mexico',
 'NA',
 'NA',
 'Japan',
 'NA',
 'America',
 'Japan',
 'Mexico',
 'America',
 'Japan',
 'America',
 'America']

In [376]:
def Proof(link):
    soup = GetSoup(link)
    el = soup.find('tr', attrs={'class':'woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_proof'})
    ele = el.text if el else "NA"
    return ele.replace('Proof','')
[Proof(link) for link in booze]

['70 ',
 '100 ',
 '100 ',
 '29.5 ',
 'NA',
 '106.2',
 '85 ',
 'NA',
 '90 ',
 '80 ',
 '76 ',
 '90.4 ',
 'NA',
 '83.4',
 '30 ',
 '80 ',
 'NA',
 '94 ',
 'NA',
 '90 ',
 '80 ',
 '107',
 '80 ',
 '100 ',
 '104 ',
 '110.2 ',
 '70 ',
 'NA',
 '70 ',
 '83.4',
 '100 ',
 'NA',
 '86 ',
 '80 ',
 'NA',
 '98.6 ',
 '80 ',
 'NA',
 'NA',
 '96 ',
 'NA',
 '90 ',
 '102 ',
 '80 ',
 '117 ',
 '86 ',
 'NA',
 '86.6 ']

In [377]:
def Availability(link):
    soup = GetSoup(link)
    el = soup.find('div', attrs={'class':'avada-availability'})
    return el.text if el else "NA"
[Availability(link) for link in booze]

['In stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'In stock',
 'In stock',
 'Only 3 left in stock',
 'In stock',
 'Out of stock',
 'In stock',
 'In stock',
 'In stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'In stock',
 'In stock',
 'Out of stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'Out of stock',
 'In stock',
 'Out of stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'Only 3 left in stock',
 'Only 3 left in stock',
 'In stock',
 'Out of stock',
 'Out of stock',
 'Out of stock',
 'In stock']